In [ ]:
from math import factorial
from scipy.stats import beta
from skfda.misc import inner_product, inner_product_matrix
from skfda.ml.regression import LinearRegression
from skfda.representation.basis import FourierBasis, FDataBasis
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Define the data
sample_size = 3
n_basis = 7
if n_basis % 2 == 0:
    n_basis = n_basis + 1
ones_sample_size = np.ones(shape=(sample_size, 1))
ones_n_basis = np.ones(shape=(n_basis, 1))
domain_range = (0, 1)
period = 1
x_basis = FourierBasis(n_basis=n_basis, domain_range=domain_range, period=period)
np.random.seed(123)
normal_sample = np.random.normal(size=(sample_size, n_basis))
coefficients_exp = np.array([1/(2 ** i) for i in range(1, n_basis + 1)]).reshape(1, -1)
coefficients_exp_matrix = np.dot(ones_sample_size, coefficients_exp)
coefficients = np.multiply(normal_sample, coefficients_exp_matrix)

In [ ]:
covariate = FDataBasis(basis=x_basis, coefficients=coefficients)

In [ ]:
n_plot = covariate.plot()

In [ ]:
# Beta distribution
cnt = 20
alpha_1 =  1 * cnt
beta_1 = 4 * cnt
rv_1 = beta(alpha_1, beta_1)

In [ ]:
# Transfor beta to fda object
n_points_abscissa_axis = 11
distance_between_points = (domain_range[1] - domain_range[0])/(n_points_abscissa_axis-1)
abscissa_points = np.arange(domain_range[0], domain_range[1] + distance_between_points, distance_between_points)
f_x = rv_1.pdf(abscissa_points)
beta_fda = FDataBasis.from_data(
    data_matrix=f_x,
    grid_points=abscissa_points,
    basis=x_basis
)
n_plot = beta_fda.plot()

In [ ]:
# Exploring the beta representation
print(beta_fda.coefficients)

In [ ]:
# Getting target variable
ration_signal_noise = 0.01
mx = inner_product(covariate, beta_fda)
var_mx = np.var(mx)
np.random.seed(123)
scale_noise = np.sqrt(ration_signal_noise * var_mx)
error_y = np.random.normal(size=sample_size, loc = 0, scale=ration_signal_noise)
print(scale_noise)
y = mx + error_y
plt.plot(mx, y, 'o')

In [ ]:
# Fit the model
linear_reg = LinearRegression()
_ = linear_reg.fit(covariate, y)
beta_estimated = linear_reg.coef_[0]
print(beta_estimated)
n_plot = beta_estimated.plot()

In [ ]:
# Predict
print(linear_reg.predict(covariate))
print(y)

In [ ]:
# Creating data for the algorithm
X = covariate(abscissa_points)
print(X.shape)
X_copy = np.squeeze(X)
print(X_copy.shape)